In [1]:
import numpy as np
import pandas as pd

import tarfile
import gzip
import re
import os
import datetime as dt

In [2]:
year_num = 5 # Number of past years to consider
extremes_num = 10 # Number of hottest and coldest places to display

In [3]:
yearfiles = os.listdir("./gsod_all_years")
yearfiles.sort()
yearfiles = yearfiles[-year_num:]
years = [int(re.findall('\d+',yearfile)[0]) for yearfile in yearfiles]

In [4]:
station_loc = pd.read_csv('isd-history.csv')
station_loc = station_loc.replace([0.0, -999.0, -999.9],np.nan)
station_loc = station_loc[pd.notnull(station_loc['LAT']) & pd.notnull(station_loc['LON'])]
station_loc = station_loc[[int(re.findall('^\d{4}', str(end_year))[0])==max(years) for end_year in station_loc['END']]]
station_loc = station_loc[[int(re.findall('^\d{4}', str(beg_year))[0])<=min(years) for beg_year in station_loc['BEGIN']]]

In [5]:
station_loc['LBL'] = station_loc[['STATION NAME','STATE','CTRY']].apply(lambda x: x.str.cat(sep=', '), axis=1)
station_loc['ELEV_LBL'] = station_loc['ELEV(M)'].apply(lambda x: 'Elevation: '+str(x)+' m' if ~np.isnan(x) else np.nan)
station_loc['LBL'] = station_loc[['LBL','ELEV_LBL']].apply(lambda x: x.str.cat(sep='<br>'), axis=1)
# station_loc = station_loc.drop(['STATION NAME','STATE','ELEV_LBL','ICAO','BEGIN','END'], axis=1)
station_loc = station_loc.drop(['ELEV_LBL','ICAO','BEGIN','END'], axis=1)
#station_loc = station_loc.sample(stat_num)

In [6]:
# Find US Stations
station_loc.head()
us_stations = station_loc.loc[station_loc['CTRY'] == 'US', :]
us_stations_list = us_stations['USAF'].tolist()

In [7]:
df = pd.DataFrame([])
df_day = pd.DataFrame([])

def preprocess_station_file_content(content):
    headers=content.pop(0)
    headers=[headers[ind] for ind in [0,1,2,3,4,8,11,12]]
    for d in range(len(content)):
        content[d]=[content[d][ind] for ind in [0,1,2,3,5,13,17,18]]
    content=pd.DataFrame(content, columns=headers)
    content.rename(columns={'STN---': 'USAF'}, inplace=True)
    content['MAX'] = content['MAX'].apply(lambda x: re.sub("\*$","",x))
    content['MIN'] = content['MIN'].apply(lambda x: re.sub("\*$","",x))
    content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']] = content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']].apply(pd.to_numeric)
    content['YEARMODA']=pd.to_datetime(content['YEARMODA'], format='%Y%m%d', errors='ignore')
    content['YEAR']=pd.DatetimeIndex(content['YEARMODA']).year
    content['MONTH']=pd.DatetimeIndex(content['YEARMODA']).month
    content['DAY']=pd.DatetimeIndex(content['YEARMODA']).day
    return content

In [8]:
# yearfile = yearfiles[-1]
# print(yearfile)
# i=0
# tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
# print(len(tar.getmembers()[1:]))
# #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
# for member in tar.getmembers()[1:]:
#     name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
#     usaf = name_parts[0]
#     wban = int(name_parts[1])
#     if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0:
#         i=i+1
#         #if i%(stat_num//10) == 0: print(i)
#         f=tar.extractfile(member)
#         f=gzip.open(f, 'rb')
#         content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
#         content=preprocess_station_file_content(content)
#         df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
#         content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
#         df = df.append(content)
# tar.close()

In [9]:
# df

In [10]:
# station_loc

In [11]:
# merged_df = pd.merge(df, station_loc, how="right", on="USAF")
# merged_df

In [12]:
# merged_clean_df = merged_df.loc[merged_df["USAF"] != "999999", :]
# merged_clean_df

In [13]:
# merged_clean_df.to_csv('merged_cleaned_dataframe.csv',index=False)

In [14]:
# day = df_day['YEARMODA'].max()
# df_day = df_day[df_day['YEARMODA']==day]

In [15]:
for yearfile in yearfiles[:-1]:
    print(yearfile)
    i=0
    tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
    print(len(tar.getmembers()[1:]))
    #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
    for member in tar.getmembers()[1:]:
        name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
        usaf = name_parts[0]
        wban = int(name_parts[1])
        if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0 and (usaf in us_stations_list) and (usaf != 999999):
            i=i+1
            #if i%(stat_num//10) == 0: print(i)
            f=tar.extractfile(member)
            f=gzip.open(f, 'rb')
            content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
            content=preprocess_station_file_content(content)
#             df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
            df_day = df_day.append(content, ignore_index=True)
            content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
            df = df.append(content)
    tar.close()

gsod_2015.tar
11664
gsod_2016.tar
12076
gsod_2017.tar
11923
gsod_2018.tar
12875


In [16]:
df_day

,USAF,WBAN,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY
0,720561,467,2015-01-01,28.8,15.5,0.6,32.9,25.3,2015,1,1
1,720561,467,2015-01-02,34.5,32.4,0.7,38.1,32.0,2015,1,2
2,720561,467,2015-01-03,38.7,36.7,3.0,43.3,36.9,2015,1,3
3,720561,467,2015-01-04,30.3,19.1,8.4,43.3,17.6,2015,1,4
4,720561,467,2015-01-05,25.6,9.2,4.8,44.6,15.8,2015,1,5
...,...,...,...,...,...,...,...,...,...,...,...
3288833,720348,63886,2018-12-27,52.3,41.5,6.5,60.8,48.2,2018,12,27
3288834,720348,63886,2018-12-28,63.6,61.6,3.8,71.6,57.2,2018,12,28
3288835,720348,63886,2018-12-29,59.1,56.2,1.3,68.0,53.6,2018,12,29
3288836,720348,63886,2018-12-30,55.4,54.8,0.6,59.0,53.6,2018,12,30


In [17]:
station_loc

,USAF,WBAN,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
11,010010,99999,JAN MAYEN(NOR-NAVY),NO,NaN,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
13,010014,99999,SORSTOKKEN,NO,NaN,59.792,5.341,48.8,"SORSTOKKEN, NO<br>Elevation: 48.8 m"
17,010020,99999,VERLEGENHUKEN,NO,NaN,80.050,16.250,8.0,"VERLEGENHUKEN, NO<br>Elevation: 8.0 m"
18,010030,99999,HORNSUND,NO,NaN,77.000,15.500,12.0,"HORNSUND, NO<br>Elevation: 12.0 m"
21,010060,99999,EDGEOYA,NO,NaN,78.250,22.817,14.0,"EDGEOYA, NO<br>Elevation: 14.0 m"
...,...,...,...,...,...,...,...,...,...
29770,A07355,241,VIROQUA MUNICIPAL AIRPORT,US,WI,43.579,-90.913,394.1,"VIROQUA MUNICIPAL AIRPORT, WI, US<br>Elevation..."
29771,A07357,182,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRPORT,US,MN,45.986,-95.992,367.3,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRP...
29772,A07359,240,IONIA COUNTY AIRPORT,US,MI,42.938,-85.061,249.0,"IONIA COUNTY AIRPORT, MI, US<br>Elevation: 249..."
29773,A51255,445,DEMOPOLIS MUNICIPAL AIRPORT,US,AL,32.464,-87.954,34.1,"DEMOPOLIS MUNICIPAL AIRPORT, AL, US<br>Elevati..."


In [18]:
station_loc['WBAN'] = station_loc['WBAN'].astype(float)
station_loc

,USAF,WBAN,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
11,010010,99999.0,JAN MAYEN(NOR-NAVY),NO,NaN,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
13,010014,99999.0,SORSTOKKEN,NO,NaN,59.792,5.341,48.8,"SORSTOKKEN, NO<br>Elevation: 48.8 m"
17,010020,99999.0,VERLEGENHUKEN,NO,NaN,80.050,16.250,8.0,"VERLEGENHUKEN, NO<br>Elevation: 8.0 m"
18,010030,99999.0,HORNSUND,NO,NaN,77.000,15.500,12.0,"HORNSUND, NO<br>Elevation: 12.0 m"
21,010060,99999.0,EDGEOYA,NO,NaN,78.250,22.817,14.0,"EDGEOYA, NO<br>Elevation: 14.0 m"
...,...,...,...,...,...,...,...,...,...
29770,A07355,241.0,VIROQUA MUNICIPAL AIRPORT,US,WI,43.579,-90.913,394.1,"VIROQUA MUNICIPAL AIRPORT, WI, US<br>Elevation..."
29771,A07357,182.0,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRPORT,US,MN,45.986,-95.992,367.3,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRP...
29772,A07359,240.0,IONIA COUNTY AIRPORT,US,MI,42.938,-85.061,249.0,"IONIA COUNTY AIRPORT, MI, US<br>Elevation: 249..."
29773,A51255,445.0,DEMOPOLIS MUNICIPAL AIRPORT,US,AL,32.464,-87.954,34.1,"DEMOPOLIS MUNICIPAL AIRPORT, AL, US<br>Elevati..."


In [19]:
merged_df = pd.merge(df_day, station_loc, how="outer", on="USAF")
merged_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,720561,467.0,2015-01-01,28.8,15.5,0.6,32.9,25.3,2015.0,1.0,1.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
1,720561,467.0,2015-01-02,34.5,32.4,0.7,38.1,32.0,2015.0,1.0,2.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
2,720561,467.0,2015-01-03,38.7,36.7,3.0,43.3,36.9,2015.0,1.0,3.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
3,720561,467.0,2015-01-04,30.3,19.1,8.4,43.3,17.6,2015.0,1.0,4.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
4,720561,467.0,2015-01-05,25.6,9.2,4.8,44.6,15.8,2015.0,1.0,5.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47820080,A07355,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241.0,VIROQUA MUNICIPAL AIRPORT,US,WI,43.579,-90.913,394.1,"VIROQUA MUNICIPAL AIRPORT, WI, US<br>Elevation..."
47820081,A07357,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,182.0,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRPORT,US,MN,45.986,-95.992,367.3,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRP...
47820082,A07359,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240.0,IONIA COUNTY AIRPORT,US,MI,42.938,-85.061,249.0,"IONIA COUNTY AIRPORT, MI, US<br>Elevation: 249..."
47820083,A51255,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,445.0,DEMOPOLIS MUNICIPAL AIRPORT,US,AL,32.464,-87.954,34.1,"DEMOPOLIS MUNICIPAL AIRPORT, AL, US<br>Elevati..."


In [20]:
merged_clean_df = merged_df.loc[merged_df['MIN'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df['MAX'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df['TEMP'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df["USAF"] != "999999", :]
merged_clean_df.dropna(how='any')
us_merged_df = merged_clean_df.loc[merged_clean_df["CTRY"] == "US", :]
us_merged_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,720561,467.0,2015-01-01,28.8,15.5,0.6,32.9,25.3,2015.0,1.0,1.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
1,720561,467.0,2015-01-02,34.5,32.4,0.7,38.1,32.0,2015.0,1.0,2.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
2,720561,467.0,2015-01-03,38.7,36.7,3.0,43.3,36.9,2015.0,1.0,3.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
3,720561,467.0,2015-01-04,30.3,19.1,8.4,43.3,17.6,2015.0,1.0,4.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
4,720561,467.0,2015-01-05,25.6,9.2,4.8,44.6,15.8,2015.0,1.0,5.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47820080,A07355,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241.0,VIROQUA MUNICIPAL AIRPORT,US,WI,43.579,-90.913,394.1,"VIROQUA MUNICIPAL AIRPORT, WI, US<br>Elevation..."
47820081,A07357,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,182.0,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRPORT,US,MN,45.986,-95.992,367.3,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRP...
47820082,A07359,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240.0,IONIA COUNTY AIRPORT,US,MI,42.938,-85.061,249.0,"IONIA COUNTY AIRPORT, MI, US<br>Elevation: 249..."
47820083,A51255,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,445.0,DEMOPOLIS MUNICIPAL AIRPORT,US,AL,32.464,-87.954,34.1,"DEMOPOLIS MUNICIPAL AIRPORT, AL, US<br>Elevati..."


In [21]:
new_df=us_merged_df.dropna(how='any')

In [22]:
new_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,720561,467.0,2015-01-01,28.8,15.5,0.6,32.9,25.3,2015.0,1.0,1.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
1,720561,467.0,2015-01-02,34.5,32.4,0.7,38.1,32.0,2015.0,1.0,2.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
2,720561,467.0,2015-01-03,38.7,36.7,3.0,43.3,36.9,2015.0,1.0,3.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
3,720561,467.0,2015-01-04,30.3,19.1,8.4,43.3,17.6,2015.0,1.0,4.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
4,720561,467.0,2015-01-05,25.6,9.2,4.8,44.6,15.8,2015.0,1.0,5.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47810213,720724,99999.0,2018-12-28,24.2,22.5,4.7,29.1,21.2,2018.0,12.0,28.0,99999.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810214,720724,99999.0,2018-12-29,10.5,6.9,5.0,21.7,-1.3,2018.0,12.0,29.0,99999.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810215,720724,99999.0,2018-12-30,10.5,6.5,4.0,23.2,-2.2,2018.0,12.0,30.0,99999.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810216,720724,99999.0,2018-12-31,13.1,9.4,3.4,28.4,2.7,2018.0,12.0,31.0,99999.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"


In [23]:
# us_drop_na_df = us_merged_df.dropna(how='any')
# us_drop_na_df

In [24]:
new_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,720561,467.0,2015-01-01,28.8,15.5,0.6,32.9,25.3,2015.0,1.0,1.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
1,720561,467.0,2015-01-02,34.5,32.4,0.7,38.1,32.0,2015.0,1.0,2.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
2,720561,467.0,2015-01-03,38.7,36.7,3.0,43.3,36.9,2015.0,1.0,3.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
3,720561,467.0,2015-01-04,30.3,19.1,8.4,43.3,17.6,2015.0,1.0,4.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
4,720561,467.0,2015-01-05,25.6,9.2,4.8,44.6,15.8,2015.0,1.0,5.0,467.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47810213,720724,99999.0,2018-12-28,24.2,22.5,4.7,29.1,21.2,2018.0,12.0,28.0,99999.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810214,720724,99999.0,2018-12-29,10.5,6.9,5.0,21.7,-1.3,2018.0,12.0,29.0,99999.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810215,720724,99999.0,2018-12-30,10.5,6.5,4.0,23.2,-2.2,2018.0,12.0,30.0,99999.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810216,720724,99999.0,2018-12-31,13.1,9.4,3.4,28.4,2.7,2018.0,12.0,31.0,99999.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"


In [25]:
if (new_df['WBAN_x'].equals(new_df['WBAN_y'])):
    new_df = new_df.drop(['WBAN_y'], axis=1)
    new_df = new_df.rename(columns={"WBAN_x": "WBAN"})
new_df


,USAF,WBAN,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,720561,467.0,2015-01-01,28.8,15.5,0.6,32.9,25.3,2015.0,1.0,1.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
1,720561,467.0,2015-01-02,34.5,32.4,0.7,38.1,32.0,2015.0,1.0,2.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
2,720561,467.0,2015-01-03,38.7,36.7,3.0,43.3,36.9,2015.0,1.0,3.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
3,720561,467.0,2015-01-04,30.3,19.1,8.4,43.3,17.6,2015.0,1.0,4.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
4,720561,467.0,2015-01-05,25.6,9.2,4.8,44.6,15.8,2015.0,1.0,5.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47810213,720724,99999.0,2018-12-28,24.2,22.5,4.7,29.1,21.2,2018.0,12.0,28.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810214,720724,99999.0,2018-12-29,10.5,6.9,5.0,21.7,-1.3,2018.0,12.0,29.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810215,720724,99999.0,2018-12-30,10.5,6.5,4.0,23.2,-2.2,2018.0,12.0,30.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"
47810216,720724,99999.0,2018-12-31,13.1,9.4,3.4,28.4,2.7,2018.0,12.0,31.0,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"


In [26]:
# us_drop_na_df['YEARMODA']

In [27]:
# list(us_drop_na_df['YEARMODA'])

In [28]:
list(new_df['YEARMODA'])

[Timestamp('2015-01-01 00:00:00'),
 Timestamp('2015-01-02 00:00:00'),
 Timestamp('2015-01-03 00:00:00'),
 Timestamp('2015-01-04 00:00:00'),
 Timestamp('2015-01-05 00:00:00'),
 Timestamp('2015-01-06 00:00:00'),
 Timestamp('2015-01-07 00:00:00'),
 Timestamp('2015-01-08 00:00:00'),
 Timestamp('2015-01-09 00:00:00'),
 Timestamp('2015-01-10 00:00:00'),
 Timestamp('2015-01-11 00:00:00'),
 Timestamp('2015-01-12 00:00:00'),
 Timestamp('2015-01-13 00:00:00'),
 Timestamp('2015-01-14 00:00:00'),
 Timestamp('2015-01-15 00:00:00'),
 Timestamp('2015-01-16 00:00:00'),
 Timestamp('2015-01-17 00:00:00'),
 Timestamp('2015-01-18 00:00:00'),
 Timestamp('2015-01-19 00:00:00'),
 Timestamp('2015-01-20 00:00:00'),
 Timestamp('2015-01-21 00:00:00'),
 Timestamp('2015-01-22 00:00:00'),
 Timestamp('2015-01-23 00:00:00'),
 Timestamp('2015-01-24 00:00:00'),
 Timestamp('2015-01-25 00:00:00'),
 Timestamp('2015-01-26 00:00:00'),
 Timestamp('2015-01-27 00:00:00'),
 Timestamp('2015-01-28 00:00:00'),
 Timestamp('2015-01-

In [29]:

new_df.dtypes

USAF                    object
WBAN                   float64
YEARMODA        datetime64[ns]
TEMP                   float64
DEWP                   float64
WDSP                   float64
MAX                    float64
MIN                    float64
YEAR                   float64
MONTH                  float64
DAY                    float64
STATION NAME            object
CTRY                    object
STATE                   object
LAT                    float64
LON                    float64
ELEV(M)                float64
LBL                     object
dtype: object

In [30]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.USWeather
collection = db.weather
db.collection.insert_many(new_df.to_dict('records'))

In [31]:
merged_agg_df = pd.merge(df, station_loc, how="outer", on="USAF")
merged_agg_df
merged_clean_agg_df = merged_agg_df.loc[merged_agg_df['MIN'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df['MAX'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df['TEMP'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df["USAF"] != "999999", :]
merged_clean_agg_df.dropna(how='any')
us_merged_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df["CTRY"] == "US", :]
us_merged_agg_df
new_agg_df=us_merged_agg_df.dropna(how='any')
if (new_agg_df['WBAN_x'].equals(new_agg_df['WBAN_y'])):
    new_agg_df = new_agg_df.drop(['WBAN_y'], axis=1)
    new_agg_df = new_agg_df.rename(columns={"WBAN_x": "WBAN"})
new_agg_df

,USAF,WBAN,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,720561,467.0,2015.0,1.0,40.30,25.70,4.60,50.9,32.00,16.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
1,720561,467.0,2015.0,2.0,37.75,20.80,5.75,52.7,26.70,14.5,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
2,720561,467.0,2015.0,3.0,55.70,42.40,5.40,70.0,46.40,16.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
3,720561,467.0,2015.0,4.0,64.60,50.85,6.95,78.8,54.85,16.5,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
4,720561,467.0,2015.0,5.0,68.80,61.80,5.50,80.4,62.80,16.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588156,720359,23902.0,2018.0,10.0,57.50,49.40,3.40,71.6,50.00,16.0,SEMINOLE MUNICIPAL AIRPORT,US,OK,35.275,-96.675,312.1,"SEMINOLE MUNICIPAL AIRPORT, OK, US<br>Elevatio..."
1588157,720359,23902.0,2018.0,11.0,47.90,32.20,4.40,62.6,35.60,15.5,SEMINOLE MUNICIPAL AIRPORT,US,OK,35.275,-96.675,312.1,"SEMINOLE MUNICIPAL AIRPORT, OK, US<br>Elevatio..."
1588158,720359,23902.0,2018.0,12.0,43.40,29.00,5.90,55.4,33.80,16.0,SEMINOLE MUNICIPAL AIRPORT,US,OK,35.275,-96.675,312.1,"SEMINOLE MUNICIPAL AIRPORT, OK, US<br>Elevatio..."
1588159,720724,99999.0,2018.0,12.0,36.00,15.30,4.15,52.7,24.55,17.5,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m"


In [32]:
#Set a timestamp to the beginning of the month for aggregates
# https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns-apply-a-function-o
# https://www.w3schools.com/python/python_datetime.asp
def add_aggregate_timestamp(row):
    return dt.datetime(int(row['YEAR']), int(row['MONTH']), 1)

new_agg_df['YEARMODA'] = new_agg_df.apply(lambda row: add_aggregate_timestamp(row), axis=1)
new_agg_df

,USAF,WBAN,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL,YEARMODA
0,720561,467.0,2015.0,1.0,40.30,25.70,4.60,50.9,32.00,16.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E...",2015-01-01
1,720561,467.0,2015.0,2.0,37.75,20.80,5.75,52.7,26.70,14.5,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E...",2015-02-01
2,720561,467.0,2015.0,3.0,55.70,42.40,5.40,70.0,46.40,16.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E...",2015-03-01
3,720561,467.0,2015.0,4.0,64.60,50.85,6.95,78.8,54.85,16.5,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E...",2015-04-01
4,720561,467.0,2015.0,5.0,68.80,61.80,5.50,80.4,62.80,16.0,WILLIAM R POUGE MUNICIPAL AIRPORT,US,OK,36.175,-96.152,272.2,"WILLIAM R POUGE MUNICIPAL AIRPORT, OK, US<br>E...",2015-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588156,720359,23902.0,2018.0,10.0,57.50,49.40,3.40,71.6,50.00,16.0,SEMINOLE MUNICIPAL AIRPORT,US,OK,35.275,-96.675,312.1,"SEMINOLE MUNICIPAL AIRPORT, OK, US<br>Elevatio...",2018-10-01
1588157,720359,23902.0,2018.0,11.0,47.90,32.20,4.40,62.6,35.60,15.5,SEMINOLE MUNICIPAL AIRPORT,US,OK,35.275,-96.675,312.1,"SEMINOLE MUNICIPAL AIRPORT, OK, US<br>Elevatio...",2018-11-01
1588158,720359,23902.0,2018.0,12.0,43.40,29.00,5.90,55.4,33.80,16.0,SEMINOLE MUNICIPAL AIRPORT,US,OK,35.275,-96.675,312.1,"SEMINOLE MUNICIPAL AIRPORT, OK, US<br>Elevatio...",2018-12-01
1588159,720724,99999.0,2018.0,12.0,36.00,15.30,4.15,52.7,24.55,17.5,BELEN RGNL ARPT,US,NM,34.646,-106.836,1584.8,"BELEN RGNL ARPT, NM, US<br>Elevation: 1584.8 m",2018-12-01


In [33]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.USWeatherAgg
collection = db.weather
db.collection.insert_many(new_agg_df.to_dict('records'))